In [34]:
import requests
import pandas as pd
from IPython.core.debugger import set_trace

In [4]:
class NotProperResponseError(Exception):
    pass

class NotProperInputError(Exception):
    pass

class SomethingWrongError(Exception):
    pass

In [93]:
def request_hist_hour(nbars=None, utc_to=None, coin=None, base_ccy=None, ex=None):
    if (nbars is None) or (utc_to is None) or (coin is None) or (base_ccy is None) or (ex is None):
        raise NotProperInputError()
    
    utc_to = int(pd.Timestamp(utc_to).timestamp())
    baseurl = ('https://min-api.cryptocompare.com/data/histohour?'
               'fsym={fsym}'
               '&tsym={tsym}'
               '&e={e}'
               '&limit={limit}'
               '&toTs={toTs}')
    
    url = baseurl.format(fsym=coin, tsym=base_ccy, e=ex, limit=nbars, toTs=utc_to)
    resp = requests.get(url).json()
    msg = resp['Response']
    
    if msg == 'Success':
        truncated = True
        utc_from = None
        
        data = {str(pd.to_datetime(_r.pop('time'), unit='s')):_r for _r in resp['Data'] if _r['close'] != 0}
        
        if len(data) > 0:
            n_del = max(0, len(data) - nbars)
            keys = sorted(data)
            keys_del = keys[:n_del]
            [data.pop(k) for k in keys_del]
        
            if len(data) == nbars:
                truncated = False
                utc_from = keys[n_del]            
        
        return {
            'data': data, 
            'truncated': truncated, 
            'utc_from': utc_from, 
        }
    
    elif msg == 'Error':
        raise NotProperResponseError()
        
    else:
        raise SomethingWrongError()

In [94]:
req = request_hist_hour(nbars=2, utc_to='2018-12-26 08:30', coin='BTC', base_ccy='KRW', ex='bithumb'); req

{'data': {'2018-12-26 07:00:00': {'close': 4235000,
   'high': 4269000,
   'low': 4224000,
   'open': 4269000,
   'volumefrom': 331.31,
   'volumeto': 1405821265.57},
  '2018-12-26 08:00:00': {'close': 4215000,
   'high': 4236000,
   'low': 4192000,
   'open': 4235000,
   'volumefrom': 220.47,
   'volumeto': 929009844.99}},
 'truncated': False,
 'utc_from': '2018-12-26 07:00:00'}

In [95]:
def request_hist_hour_range(utc_from=None, utc_to=None, coin=None, base_ccy=None, ex=None):
    data = {}
    
    if utc_from is None:
        utc_from = pd.Timestamp('2000-01-01')
    else:
        utc_from = pd.Timestamp(utc_from).floor('h')
        
    if utc_to is None:
        utc_to = pd.Timestamp.utcnow().floor('h') + pd.Timedelta(-1, unit='h')
    else:
        utc_to = pd.Timestamp(utc_to).floor('h')
    
    while True:
        nbars = int((utc_to.timestamp() - utc_from.timestamp()) / 3600) + 1
        nbars = min(2000, nbars)
        req = request_hist_hour(nbars=nbars, utc_to=utc_to, coin=coin, base_ccy=base_ccy, ex=ex)
        data.update(req['data'])
        
        if req['truncated'] or (nbars < 2000): 
            break
        else:
            utc_to = pd.Timestamp(req['utc_from']) + pd.Timedelta(-1, unit='s')
            
    return data

In [104]:
# rs = request_hist_hour_range(utc_from='2019-01-26 00:00:00', utc_to='2019-06-26 03:00:00', coin='BTC', base_ccy='KRW', ex='upbit'
rs = request_hist_hour_range(coin='EOS', base_ccy='KRW', ex='upbit')

In [116]:
pd.DataFrame(rs).T.sort_index()#.iloc[5400:]

,close,high,low,open,volumefrom,volumeto
2018-04-27 22:00:00,18500.0,18910.0,18480.0,18760.0,148731.64,2.787138e+09
2018-04-27 23:00:00,18240.0,18750.0,18040.0,18500.0,224073.20,4.129044e+09
2018-04-28 00:00:00,19080.0,19190.0,18070.0,18240.0,285270.40,5.345520e+09
2018-04-28 01:00:00,19360.0,19420.0,19000.0,19080.0,305055.82,5.866705e+09
2018-04-28 02:00:00,18910.0,19360.0,18810.0,19360.0,204507.56,3.908616e+09
2018-04-28 03:00:00,19160.0,19230.0,18890.0,18910.0,250289.14,4.781831e+09
2018-04-28 04:00:00,19080.0,19170.0,19020.0,19160.0,205492.25,3.924290e+09
2018-04-28 05:00:00,19310.0,19310.0,19040.0,19080.0,152013.86,2.909048e+09
2018-04-28 06:00:00,19960.0,20200.0,19220.0,19310.0,345089.21,6.814108e+09
2018-04-28 07:00:00,19990.0,20160.0,19770.0,19960.0,310163.38,6.192218e+09


In [96]:
# url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=1'
# url = 'https://api.coingecko.com/api/v3/exchanges/bithumb'
url = 'https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=KRW&limit=1&e=upbit'
resp = requests.get(url).json(); resp

{'Response': 'Success',
 'Type': 100,
 'Aggregated': False,
 'Data': [{'time': 1561525200,
   'close': 14951000,
   'high': 15592000,
   'low': 14930000,
   'open': 14939000,
   'volumefrom': 494.6,
   'volumeto': 7513933948.41},
  {'time': 1561528800,
   'close': 15119000,
   'high': 15153000,
   'low': 14951000,
   'open': 14951000,
   'volumefrom': 192.89,
   'volumeto': 2904418221.13}],
 'TimeTo': 1561528800,
 'TimeFrom': 1561525200,
 'FirstValueInArray': True,
 'ConversionType': {'type': 'force_direct', 'conversionSymbol': ''},
 'RateLimit': {},
 'HasWarning': False}

In [75]:
prices = pd.DataFrame(resp['Data'])
prices

,close,high,low,open,time,volumefrom,volumeto
0,14286000,14410000,14178000,14304000,1561510800,367.01,5.249753e+09
1,14300000,14307000,14043000,14286000,1561514400,428.64,6.082490e+09
2,14697000,14699000,14279000,14300000,1561518000,423.25,6.105994e+09
3,14842000,14842000,14571000,14697000,1561521600,321.21,4.710727e+09
